In [1]:
from chords_and_notes_dict import * 
import threading 
import tkinter as tk
from PIL import ImageTk, Image
import mido
import random 
import os
from functools import partial
import pickle
import time
from datetime import datetime
import copy
import rtmidi as rt

# Phases de test

('2- KOMPLETE KONTROL - 1 3', 'Microsoft GS Wavetable Synth 0')

# Script

In [19]:
def open_port(input_name: str, output_name: str): 
    """ 
    Input_port: MIDI controller to send MIDI messages 
    Output_port: Corresponds to the sound that the programm returns after getting MIDI message --> it can be no sound 

    Case 1: If input and output ports are available --> open them with the names given in parameters
    Case 2: If input and output ports are already used --> close them and open them with the names given in parameters

    It returns the input_port and output_port 
    """

    global input_port, output_port

    #if input_port and output_port are AVAILABLE
    try: 
        input_port = mido.open_input(input_name)
        output_port = mido.open_output(output_name)

    #if input_port and output_port are NOT AVAILABLE    
    except Exception:
        close_port(input_port, output_port)
        input_port = mido.open_input(input_name)
        output_port = mido.open_output(output_name)
    
    return input_port, output_port 


def close_port(input_port, output_port) -> None: 
    """ 
    Close input and output ports
    """
    input_port.close()
    output_port.close()


def choose_ports():
    """ 
    Create a Tkinter window in which the input and output names are displayed on your machine
    
    - Select ONLY one for the input by clicking inside a checkbox in the INPUTS column
    - Select ONLY one for the ouput by clicking inside a checkbox in the OUTPUTS column

    It returns the input_name and output_name selected
    """

    def validate_checkboxes():
        """ 
        Retrieve the names of the input and output selected by clicking inside their checkbox
        Then it kills the Tkinter "Inputs & Outputs" window
        """

        global selected_input, selected_output

        input_index = [var.get() for var in input_vars].index(1)
        selected_input = input_options[input_index]
        output_index = [var.get() for var in output_vars].index(1)
        selected_output = output_options[output_index]

        print(f"Input selected: '{selected_input}'")
        print(f"Output selected: '{selected_output}'")

        window.destroy()


    window = tk.Tk()
    window.geometry("600x400")

    input_label = tk.Label(window, text="INPUTS", font=("Aerial", 20))
    input_label.grid(row=0, column=0, sticky="w")

    output_label = tk.Label(window, text="OUTPUTS", font=("Aerial", 20))
    output_label.grid(row=0, column=1, sticky="w")

    # Create the lists of input and output options available on your machine
    input_options = mido.get_input_names()
    output_options = mido.get_output_names()

    # Create variables to track the state of the checkboxes
    input_vars = []
    output_vars = []

    # Create checkboxes for the input options
    for i, option in enumerate(input_options):
        var = tk.IntVar()
        checkbox = tk.Checkbutton(window, text=option, variable=var)
        checkbox.grid(row=i+1, column=0, sticky="w")
        input_vars.append(var)

    # Create checkboxes for the output options
    for i, option in enumerate(output_options):
        var = tk.IntVar()
        checkbox = tk.Checkbutton(window, text=option, variable=var)
        checkbox.grid(row=i+1, column=1, sticky="w")
        output_vars.append(var)

    # Create the entry button to validate the checkboxes
    button = tk.Button(window, text="Validate", command=validate_checkboxes)
    button.grid(row=max(len(input_options), len(output_options))+1, columnspan=2)

    window.mainloop()

    return selected_input, selected_output




In [20]:
input_selected, output_selected = choose_ports()

Input selected: '2- KOMPLETE KONTROL - 1 2'
Output selected: 'Microsoft GS Wavetable Synth 0'


In [ ]:
open_port()

In [7]:
open_port(input_piano, output_piano)

ok
ok


(<open input '2- KOMPLETE KONTROL - 1 2' (RtMidi/WINDOWS_MM)>,
 <open output 'Microsoft GS Wavetable Synth 0' (RtMidi/WINDOWS_MM)>)